**SA367 &#x25aa; Mathematical Models for Decision Making &#x25aa; Spring 2022 &#x25aa; Uhan**

# Lesson 14. Solving stochastic dynamic programs with Python

## Overview

* Let's solve the stochastic dynamic program we formulated for the investment problem in Lesson 13


* In this class, we will use a package called `stochasticdp` to set up and solve stochastic dynamic programs
    - _Warning._ This is a package that I wrote. There may be some bugs.
    - _Note._ This package is publicly available. Please feel free to use it in the future for other things. The source code is on [GitHub](https://github.com/nelsonuhan/stochasticdp).


## Setting up a stochastic dynamic program

* To use `stochasticdp`, we must first import it. In `stochasticdp`, we only need the object `StochasticDP`, so we can perform our import like this:

In [ ]:
from stochasticdp import StochasticDP

* Recall the investment problem from Lesson 13:

__Problem.__ Suppose you have \\$5,000 to invest. Over the next 3 years, you want to double your money.
At the beginning of each of the next 3 years, you have an opportunity to invest in one of two investments: A or B. Both investments have uncertain profits. For an investment of \\$5,000, the profits are as follows:


| Investment | Profit (\$) | Probability |
|:-----------|------------:|------------:|
| A          | -5,000      | 0.3         |
|            | 5,000       | 0.7         |
| B          | 0           | 0.9         |
|            | 5,000       | 0.1         |
                                     
You are allowed to make at most one investment each year, and can invest only \\$5,000 each time. Any additional money accumulated is left idle. Once you've accumulated \\$10,000, you stop investing.

Formulate a stochastic dynamic program to find an investment policy that maximizes the probability you will have \$10,000 after 3 years.

* Let's walk through setting up the stochastic DP we formulated for this problem

### Initialization

* We had defined 4 stages


* To make things easier, let's renumber the stages so they start at $t = 0$:

$$
\begin{aligned}
\text{stage } t = 0, 1, 2 & \quad\leftrightarrow\quad \text{beginning of year $t$}\\
t = 3 & \quad\leftrightarrow\quad \text{end of process}
\end{aligned}
$$
    

* In each stage, we defined 3 states:

$$
\text{state } n \in \{0, 5000, 10000\} \quad\leftrightarrow\quad \text{$n$ dollars in account}
$$


* At each stage and state, we defined 3 possible decisions:

$$
\text{decision } x_t \in \{ \text{A}, \text{B}, \text{no investment} \}
$$


* The set of _allowable_ decisions changed, depending on the stage and state &mdash; we'll address this later


* For now, we can initialize a stochastic dynamic program with these stages, states, and decisions like this:

* The code above initializes a stochastic dynamic program called `dp`

* Next, we need to add every transition that occurs with positive probability

### Transition probabilities and contributions

* First, let's tackle transitions from the state $n = 5000$:

<center><img src="img/5000.png" width="600"></center>

* We can add a transition from state $n$ to state $m$ in stage $t$ under decision $x$ with probability $p(m \,|\, n, t, x)$ and contribution $c(m \,|\, n, t, x)$ as follows:

    ```python
    dp.add_transition(stage=t, from_state=n, decision=x, to_state=n, probability=p, contribution=c)
    ```


* Remember that the contributions for all transitions are 0 in this stochastic DP


* So, we can input the transition probabilities and contributions from state $n = 5000$ in stages $t = 0, 1, 2$ as follows:

* _Quick check._ What can the sum of the transition probabilities from any decision node equal?

_Write your notes here. Double-click to edit._

* Next, let's tackle the transitions from state $n = 0$:

<center><img src="img/0.png" width="600"></center>

* So, we can input the transition probabilities and contributions from state $n = 0$ in stages $t = 0, 1, 2$ like this:

* We can tackle the transitions from state $n = 10000$ in an almost identical way:

<center><img src="img/10000.png" width="600"></center>

### Boundary conditions

* Finally, we need to define the boundary conditions:

$$
f_3(10000) = 1 \qquad f_3(5000) = 0 \qquad f_3(0) = 0
$$

* In particular, we need to specify the value-to-go function at the last stage (in our case, $t = 3$) for each state

* We can add the boundary conditions for state $n$ like this:

```python
dp.add_boundary(state=0, value=0)
```

* So, let's add the boundary conditions for our problem:

## Solving the stochastic dynamic program

* Once the stochastic DP is setup, we can solve it like this:

* Note that the method `.solve()` outputs two objects: `value` and `policy`


* `value[t, n]` is the value-to-go function $f_t(n)$ at stage $t$ and state $n$


* `policy[t, n]` is the optimal decision $x_t^*$ that attains the value-to-go function $f_t(n)$ at stage $t$ and state $n$

* First, let's see what the value-to-go function looks like:

In [ ]:
# Examine the value-to-go function
print(value)

* Next, let's look at the corresponding policy:

In [ ]:
# Examine the policy
print(policy)

## On your own

__Problem.__ The Hit-and-Miss Manufacturing Company has received an order to supply one item of a particular type. However, manufacturing this item is difficult, and the customer has specified such stringent quality requirements that the company may have to produce more than one item to obtain an item that is acceptable.
                
The company estimates that each item of this type will be acceptable with probability 1/2 and defective with probability 1/2. Each item costs \\$100 to produce, and excess items are worthless. In addition, a setup cost of \\$300 must be incurred whenever the production process is setup for this item. The company has time to make no more than 3 production runs, and at most 5 items can be produced in each run. If an acceptable item has not been obtained by the end of the third production run, the manufacturer is in breach of contract and must pay a penalty of \\$1600.
                
The objective is to determine how many items to produce in each production run in order to minimize the total expected cost.

Solve the stochastic DP we formulated in Lesson 13 for this problem.

---

## Problems

### Problem 1 (Baytheon, revisited)

Baytheon has received an order to supply 2 guided missiles. In order to meet stringent quality requirements, the company may have to manufacture more than one missile to obtain an missile that is acceptable. The company has time to make no more than 3 production runs, and at most 2 missiles can be produced in each run. The probability distribution of acceptable missiles in a given run depends on how many missiles are produced:

| Number of missiles produced | Prob. of 0 acceptable missiles | Prob. of 1 acceptable missile | Prob. of 2 acceptable missiles |
|-----------------------------|--------------------------------|-------------------------------|--------------------------------| 
| 0                           | 1                              | 0                             | 0                              |
| 1                           | 1/3                            | 2/3                           | 0                              |
| 2                           | 1/4                            | 1/2                           | 1/4                            |
    

Each missile costs \\$100,000 to produce, and excess missiles are worthless. In addition, a setup cost of \\$50,000 must be incurred whenever the production process is setup for this item. If 2 acceptable missiles have not been obtained by the end of the third production run, Baytheon is in breach of contract and must pay a penalty of \\$1,000,000. The objective is to determine how many missiles to produce in each production run in order to minimize the total expected cost.

Once upon a time, for homework, you formulated this problem as a stochastic dynamic program. (Note that the penalty value has changed.)

1. Solve your dynamic program using `stochasticdp`.
2. Interpret the output of your stochastic dynamic program.

_Write your notes here. Double-click to edit._

### Problem 2 (Farkas Investments, revisited)

You have recently been hired as a junior analyst at Farkas Investments. You have been given \$4 million to invest over the next 3 years. At the beginning of each of the next 3 years, you can invest in one of two investments: A or B.

| Investment | Cost (\$ millions) | Profit (\$ millions) | Probability |
|:-----------|:-------------------|:---------------------|:------------|
| A          | 3                  | 2                    | 0.5         |
|            |                    | -2                   | 0.5         |
| B          | 5                  | 3                    | 0.1         |
|            |                    | -1                   | 0.9         |

You are allowed to make at most one investment each year. Any additional money accumulated is left idle. You may not borrow money to invest; that is, you cannot buy into an investment if it costs more than you currently have.

Formulate a stochastic dynamic program to find an investment policy that maximizes the probability you will have at least \$10 million at the end of 3 years.

Once upon a time, for homework, you formulated this problem as a stochastic dynamic program.

1. Solve your dynamic program using `stochasticdp`.
2. Interpret the output of your stochastic dynamic program.

_Write your notes here. Double-click to edit._